# Coffee shops near London Underground stations

In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import requests

import matplotlib.cm as cm
import matplotlib.colors as colors

## Retrieiving and cleaning the basic station data

Download list of London Underground stations from Wikipedia.  Rename the `Station` column as `Name` for consistency with the OpenStreetMap table we are going to use.  Remove some extraneous characters from the column names.

In [2]:
tube_stations = pd.read_html('https://en.wikipedia.org/wiki/List_of_London_Underground_stations')[0]
tube_stations = tube_stations.rename(columns={'Station': 'Name', 'Zone(s)[†]': 'Zone', 'Usage[5]': 'Usage'})
tube_stations

,Name,Photograph,Line(s)[*],Local authority,Zone,Opened[4],Main lineopened,Other name(s)[note 2],Usage
0,Acton Town,NaN,DistrictPiccadilly,Ealing,3,1 July 1879,NaN,Mill Hill Park: 1879–1910,6.19
1,Aldgate,NaN,Metropolitan[a]Circle,City of London,1,18 November 1876,NaN,NaN,9.96
2,Aldgate East,NaN,Hammersmith & City[d]District,Tower Hamlets,1,6 October 1884resited 31 October 1938,NaN,Commercial Road: Proposed before opening,14.15
3,Alperton,NaN,Piccadilly[h],Brent,4,28 June 1903,NaN,Perivale-Alperton: 1903–10,2.86
4,Amersham,NaN,Metropolitan,Buckinghamshire,9,1 September 1892,NaN,Amersham: 1892–1922Amersham & Chesham Bois: 19...,2.35
...,...,...,...,...,...,...,...,...,...
265,Wimbledon Park,NaN,District,Merton,3,3 June 1889,NaN,NaN,2.15
266,Wood Green,NaN,Piccadilly,Haringey,3,19 September 1932,NaN,Lordship Lane: Proposed before opening,12.13
267,Wood Lane,NaN,Hammersmith & CityCircle,Hammersmith and Fulham,2,12 October 2008,NaN,NaN,4.74
268,Woodford,NaN,Central,Redbridge,4,14 December 1947,22 August 1856,NaN,5.86


Extract only the stations in Zone 1.  Note that some stations lie on the Zone 1/2 boundary and are listed in the table as `1 & 2`.  We will include these too.

In [3]:
zone1_stations = tube_stations[tube_stations['Zone'].str.contains('1')]
zone1_stations

,Name,Photograph,Line(s)[*],Local authority,Zone,Opened[4],Main lineopened,Other name(s)[note 2],Usage
1,Aldgate,NaN,Metropolitan[a]Circle,City of London,1,18 November 1876,NaN,NaN,9.96
2,Aldgate East,NaN,Hammersmith & City[d]District,Tower Hamlets,1,6 October 1884resited 31 October 1938,NaN,Commercial Road: Proposed before opening,14.15
5,Angel,NaN,Northern,Islington,1,17 November 1901,NaN,NaN,17.71
9,Baker Street,NaN,Metropolitan[b]BakerlooCircleJubileeHammersmit...,City of Westminster,1,10 January 1863,NaN,NaN,28.07
11,Bank,NaN,Waterloo & CityNorthernCentral,City of London,1,8 August 1898,NaN,City (W&C line): 1898–1940Lombard Street (Nort...,61.79[note 3]
...,...,...,...,...,...,...,...,...,...
240,Vauxhall,NaN,Victoria,Lambeth,1 & 2,23 July 1971,NaN,NaN,32.30
241,Victoria,NaN,District[i]CircleVictoria,City of Westminster,1,24 December 1868,NaN,NaN,85.47
244,Warren Street,NaN,NorthernVictoria,Camden,1,22 June 1907,NaN,Euston Road: 1907–08,18.25
246,Waterloo,NaN,Waterloo & CityBakerlooNorthernJubilee,Lambeth,1,10 March 1906,NaN,NaN,82.93


The only data we still need is the station name and usage figure, so discard the other columns.

In [4]:
zone1_stations = zone1_stations[['Name', 'Usage']]

Some stations have a footnote reference in the `Usage` column.  Strip this out.

In [5]:
import re
def strip_footnote(input):
    return re.sub('\[.*\]', '', input)

zone1_stations['Usage'] = zone1_stations['Usage'].apply(strip_footnote)
with pd.option_context('display.max_rows', None):
    display(zone1_stations)

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Name,Usage
1,Aldgate,9.96
2,Aldgate East,14.15
5,Angel,17.71
9,Baker Street,28.07
11,Bank,61.79
12,Barbican,10.47
16,Bayswater,4.25
21,Blackfriars,15.53
23,Bond Street,37.49
24,Borough,5.55


`Edgware Road` in fact refers to two different stations.  In the OpenStreetMap table, suffixes are used to distinguish them.  Edit the entries here to be consistent.

In [6]:
zone1_stations.loc[69, 'Name'] = 'Edgware Road (Bakerloo Line)'
zone1_stations.loc[70, 'Name'] = 'Edgware Road (Circle Line)'

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

Get station co-ordinates from OpenStreetMap.  Keep only the relevant columns.

In [7]:
station_coordinates = pd.read_html('https://wiki.openstreetmap.org/wiki/List_of_London_Underground_stations')[0]
station_coordinates = station_coordinates[['Name', 'Latitude', 'Longitude']]
station_coordinates

,Name,Latitude,Longitude
0,Acton Town,51.502500,-0.278126
1,Acton Central,51.50883531,-0.263033174
2,Acton Central,51.50856013,-0.262879534
3,Aldgate,51.51394,-0.07537
4,Aldgate East,51.51514,-0.07178
...,...,...,...
297,Wimbledon,51.42200,-0.20544
298,Wimbledon Park,51.43391,-0.19864
299,Wood Green,51.59709,-0.10939
300,Woodford,51.60582,+0.03328


Merge with the co-ordinates data with the usage data

In [8]:
zone1_stations = zone1_stations.merge(station_coordinates, on='Name')

Some larger stations have duplicate entries relating to different parts of the station, but there is just a combined usage figure for the whole station.  Remove the duplicates.

In [9]:
zone1_stations.drop_duplicates(subset='Name', inplace=True)
with pd.option_context('display.max_rows', None):
    display(zone1_stations)

,Name,Usage,Latitude,Longitude
0,Aldgate,9.96,51.51394,-0.07537
1,Aldgate East,14.15,51.51514,-0.07178
2,Angel,17.71,51.53253,-0.10579
3,Baker Street,28.07,51.52265,-0.15704
5,Bank,61.79,51.5134047,-0.08905843
6,Barbican,10.47,51.520865,-0.097758
7,Bayswater,4.25,51.51224,-0.187569
8,Blackfriars,15.53,51.5114403,-0.10419050
9,Bond Street,37.49,51.51461,-0.14897
10,Borough,5.55,51.50095,-0.09446


## Finding coffee shops near each station

Install and import the `folium` mapping library

In [10]:
!pip install folium

import folium

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


Plot a map of the Zone 1 stations, with a circle of radius 200 metres around each one, which will be the area we search for existing coffee shops.  200 metres was chosen because in most cases it avoids overlap between nearby stations (although there are exceptions).

In [11]:
# Centre the map on Charing Cross, which is traditionally considered to be the centre of London
# Co-ordinates obtained from Wikipedia (https://en.wikipedia.org/wiki/Charing_Cross_railway_station)
charing_cross_loc = 51.508, -0.125

def add_station_to_map(map, station):
    folium.Circle([station['Latitude'], station['Longitude']], radius=200, color='blue', fill=False).add_to(map)
    folium.Marker([station['Latitude'], station['Longitude']], popup=station['Name']).add_to(map)

map_zone1_stations = folium.Map(location=charing_cross_loc, zoom_start=13)
zone1_stations.apply(lambda station : add_station_to_map(map_zone1_stations, station), axis='columns')
map_zone1_stations

Define a function that will use Foursquare to search for coffee shops within a given radius of a location.

In [13]:
def get_coffee_shops_near_location(latitude, longitude, radius=200):
    
    venues_list = []
    
    COFFEE_SHOP_ID = '4bf58dd8d48988d1e0931735'
       
    # create the API request URL
    url = f'https://api.foursquare.com/v2/venues/search?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&limit={LIMIT}&ll={latitude},{longitude}&radius={radius}&categoryId={COFFEE_SHOP_ID}'

    # make the GET request
    results = requests.get(url).json()["response"]['venues']

    # return only relevant information for each nearby venue
    for v in results:
        venues_list.append((
            v['name'], 
            v['location']['lat'], 
            v['location']['lng']))
        
    nearby_venues = pd.DataFrame(
        # [item for venue_list in venues_list for item in venue_list],
        data=venues_list,
        columns=['Name', 'Latitude', 'Longitude'])
    
    return nearby_venues

Plot the hits for the first entry in the station list (Aldgate), to check the locations look reasonable.

In [14]:
aldgate_info = zone1_stations.iloc[0]
aldgate_coffee_shops = get_coffee_shops_near_location(aldgate_info['Latitude'], aldgate_info['Longitude'])

def add_venue_to_map(map, venue):
    lat = venue['Latitude']
    lng = venue['Longitude']
    folium.Marker((lat, lng), popup=venue['Name']).add_to(map)

map_aldgate = folium.Map(location=(aldgate_info['Latitude'], aldgate_info['Longitude']), zoom_start=16)
aldgate_coffee_shops.apply(lambda venue : add_venue_to_map(map_aldgate, venue), axis='columns')
map_aldgate

Find the coffee shops near every Zone 1 station.

In [15]:
zone1_stations['No. of coffee shops'] = zone1_stations.apply(lambda station: len(get_coffee_shops_near_location(station['Latitude'], station['Longitude'])), axis='columns')
zone1_stations

,Name,Usage,Latitude,Longitude,No. of coffee shops
0,Aldgate,9.96,51.51394,-0.07537,10
1,Aldgate East,14.15,51.51514,-0.07178,12
2,Angel,17.71,51.53253,-0.10579,12
3,Baker Street,28.07,51.52265,-0.15704,12
5,Bank,61.79,51.5134047,-0.08905843,16
6,Barbican,10.47,51.520865,-0.097758,2
7,Bayswater,4.25,51.51224,-0.187569,6
8,Blackfriars,15.53,51.5114403,-0.10419050,7
9,Bond Street,37.49,51.51461,-0.14897,25
10,Borough,5.55,51.50095,-0.09446,5


## Analysing the number of coffee shops in relation to station users

Calculate the number of coffee shops per million station users near each station.

In [16]:
zone1_stations['No. of coffee shops per million'] = zone1_stations['No. of coffee shops'] / zone1_stations['Usage'].astype(float)
zone1_stations = zone1_stations.sort_values(by='No. of coffee shops per million')
zone1_stations

,Name,Usage,Latitude,Longitude,No. of coffee shops,No. of coffee shops per million
33,Lancaster Gate,6.63,51.512083,-0.175067,0,0.000000
30,Hyde Park Corner,4.44,51.50313,-0.15278,0,0.000000
59,Vauxhall,32.30,51.48603,-0.12369,5,0.154799
63,Westminster,22.56,51.50121,-0.12489,4,0.177305
62,Waterloo,82.93,51.50322,-0.11328,15,0.180875
31,Knightsbridge,16.53,51.50169,-0.16030,3,0.181488
48,Pimlico,10.81,51.489081,-0.133037,2,0.185014
6,Barbican,10.47,51.520865,-0.097758,2,0.191022
20,Euston,41.09,51.52774,-0.13303,8,0.194695
18,Elephant & Castle,19.75,51.49467,-0.10047,4,0.202532


Take the stations with a particularly low density of coffee shops (< 0.3 per million station users)

In [17]:
low_coffee_stations = zone1_stations.loc[zone1_stations['No. of coffee shops per million'] < 0.3]
low_coffee_stations

,Name,Usage,Latitude,Longitude,No. of coffee shops,No. of coffee shops per million
33,Lancaster Gate,6.63,51.512083,-0.175067,0,0.000000
30,Hyde Park Corner,4.44,51.50313,-0.15278,0,0.000000
59,Vauxhall,32.30,51.48603,-0.12369,5,0.154799
63,Westminster,22.56,51.50121,-0.12489,4,0.177305
62,Waterloo,82.93,51.50322,-0.11328,15,0.180875
31,Knightsbridge,16.53,51.50169,-0.16030,3,0.181488
48,Pimlico,10.81,51.489081,-0.133037,2,0.185014
6,Barbican,10.47,51.520865,-0.097758,2,0.191022
20,Euston,41.09,51.52774,-0.13303,8,0.194695
18,Elephant & Castle,19.75,51.49467,-0.10047,4,0.202532


Plot a map of these stations.

In [18]:
map_low_coffee_stations = folium.Map(location=charing_cross_loc, zoom_start=13)

map_low_coffee_stations = folium.Map(location=charing_cross_loc, zoom_start=13)
low_coffee_stations.apply(lambda station : add_station_to_map(map_low_coffee_stations, station), axis='columns')
map_low_coffee_stations

## Clustering the stations with a low density of coffee shops

In [19]:
from sklearn.cluster import KMeans

Peform k-means clustering of the stations in the above list.  Local knowledge was used to choose the value of _k_ that seems to give the most appropriate grouping (_k_=6).

In [20]:
low_coffee_locations = low_coffee_stations[['Latitude', 'Longitude']].values
clusters = KMeans(n_clusters=6, random_state=0).fit(low_coffee_locations)

Plot the clusters on a map.  The size of each marker is proportional to the number of stations in the cluster.

In [21]:
map_clusters = folium.Map(location=charing_cross_loc, zoom_start=13)

# add markers to map
for i in range(clusters.n_clusters):
    cluster_center = clusters.cluster_centers_[i]
    n_cluster_members = list(clusters.labels_).count(i)
    folium.CircleMarker(
        (cluster_center[0], cluster_center[1]),
        radius=5*n_cluster_members,
        popup=f'{n_cluster_members} stations',
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters)  
    
map_clusters